In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
df = pd.read_csv('pcafilewlabels.csv', header = None)


In [7]:
X2 = df.iloc[:,40:42]
X2 = X2.apply(LabelEncoder().fit_transform)

In [8]:
X_2 = df.iloc[:,0:40].join(X2)
X_2.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.556408,0.000028,0.000418,0.000036,-0.000713,-0.000724,0.001234,-0.000199,0.002731,0.012509,...,0.000281,0.000153,-0.000026,-0.000836,0.003419,0.002796,0.000497,-0.001576,0,0
1,-0.555708,0.000018,0.000419,0.000021,-0.000703,-0.000718,0.001231,-0.000156,0.002711,0.012504,...,0.000117,0.000206,0.000019,-0.001139,0.003907,0.003582,0.000851,-0.002051,0,0
2,-0.556088,0.000028,0.000417,0.000039,-0.000715,-0.000728,0.001247,-0.000215,0.002794,0.012733,...,0.000329,0.000222,0.000167,-0.000898,0.003403,0.002881,0.000560,-0.001549,0,0
3,-0.555757,0.000019,0.000425,0.000024,-0.000701,-0.000714,0.001223,-0.000171,0.002714,0.012477,...,0.000212,0.000121,-0.000067,-0.000782,0.003404,0.002836,0.000426,-0.001647,0,0
4,-0.555139,0.000005,0.000428,0.000008,-0.000681,-0.000712,0.001199,-0.000148,0.002652,0.012260,...,-0.000058,0.000433,0.000297,-0.001159,0.003651,0.003394,0.000734,-0.001825,0,0


In [15]:
X = X_2.iloc[:,0:40].values
y = X_2[41].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 451)


In [17]:
y_pred


NameError: name 'y_pred' is not defined

In [18]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [19]:
y_train = y_train.long()
y_test = y_test.long()

In [20]:
type(y_train)


torch.Tensor

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 


(torch.Size([6840, 40]),
 torch.Size([2280, 40]),
 torch.Size([6840]),
 torch.Size([2280]))

In [22]:
training_samples = TensorDataset(X_train, y_train)
test_samples = TensorDataset(X_test, y_test)

train_loader = DataLoader(training_samples, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_samples, batch_size=64, shuffle=True)

With Adam Optimizer.
Optimizer : Adam
Learning Rate : 1e-2
Number of Hidden layers : 1
Number of neurons in hidden layers : 100
Activation function(s) used : ReLU
Batch size : 64
Epochs : 500
Final loss of the last batch / neuron : 0.297
Accuracy of the model : 0.8951

In [24]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """

        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)


    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(np.random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred
      
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 40, 100, 19

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)
# making our model operate at double precision.
model = model.double()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
# since it is multiclass classification problem, we are using CrossEntropyLoss, instead of MSEloss
criterion = torch.nn.CrossEntropyLoss()

learning_rate = 1e-2 # alpha
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Adam optimizers

for epoch in range(500):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data

    if (epoch + 1) % 100 == 0:
        print('Epoch : %d,  loss: %.3f' %
              (epoch + 1, running_loss / 64))
    running_loss = 0.0

print('Finished Training')

Epoch : 100,  loss: 0.416
Epoch : 200,  loss: 0.304
Epoch : 300,  loss: 0.258
Epoch : 400,  loss: 0.200
Epoch : 500,  loss: 0.204
Finished Training


In [25]:
# pass it through the model
X_test_var = Variable(X_test, volatile=True)
scores = model(X_test_var)
_, preds = torch.max(scores, dim=1)

accuracy_score(preds.data.numpy(), y_test.numpy())

<ipython-input-25-de52482e4f77>:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  X_test_var = Variable(X_test, volatile=True)


0.9114035087719298